Read in the Data and the Schema

In [19]:
import numpy as np
import pandas as pd
from IPython import display
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
%matplotlib inline

df = pd.read_csv(r'survey_results_public_20.csv', sep = ',', error_bad_lines = False, index_col = False)
df_schem = pd.read_csv(r'survey_results_schema_20.csv', sep = ',', error_bad_lines = False, 
                       index_col = False)
df.head()

,Respondent,MainBranch,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,ConvertedComp,Country,CurrencyDesc,...,SurveyEase,SurveyLength,Trans,UndergradMajor,WebframeDesireNextYear,WebframeWorkedWith,WelcomeChange,WorkWeekHrs,YearsCode,YearsCodePro
0,1,I am a developer by profession,Yes,NaN,13,Monthly,NaN,NaN,Germany,European Euro,...,Neither easy nor difficult,Appropriate in length,No,"Computer science, computer engineering, or sof...",ASP.NET Core,ASP.NET;ASP.NET Core,Just as welcome now as I felt last year,50.0,36,27
1,2,I am a developer by profession,No,NaN,19,NaN,NaN,NaN,United Kingdom,Pound sterling,...,NaN,NaN,NaN,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat more welcome now than last year,NaN,7,4
2,3,I code primarily as a hobby,Yes,NaN,15,NaN,NaN,NaN,Russian Federation,NaN,...,Neither easy nor difficult,Appropriate in length,NaN,NaN,NaN,NaN,Somewhat more welcome now than last year,NaN,4,NaN
3,4,I am a developer by profession,Yes,25.0,18,NaN,NaN,NaN,Albania,Albanian lek,...,NaN,NaN,No,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat less welcome now than last year,40.0,7,4
4,5,"I used to be a developer by profession, but no...",Yes,31.0,16,NaN,NaN,NaN,United States,NaN,...,Easy,Too short,No,"Computer science, computer engineering, or sof...",Django;Ruby on Rails,Ruby on Rails,Just as welcome now as I felt last year,NaN,15,8


Only work with columns that have at least 70% reported data

In [20]:
def remove_nan(df, threshold):
    df = df.dropna(axis = 1,thresh = (threshold*df.shape[0]))
    return  df

df = remove_nan(df,0.7)

Remove categories irrelevant to the proejct purpose

In [21]:
to_drop = ['SOAccount','SOComm','SOPartFreq','SOVisitFreq','SurveyEase','SurveyLength','WelcomeChange','NEWOffTopic']
df = df.drop(to_drop, axis = 1)

Reduce schema to represent only columns remaining in data set

In [22]:
def remove_schema_rows(df,schema):
    col_names = list(df.columns)
    new_schema = schema[~schema.Column.isin(col_names) == False]
    return new_schema

remove_schema_rows(df,df_schem)

,Column,QuestionText
0,Respondent,Randomized respondent ID number (not in order ...
1,MainBranch,Which of the following options best describes ...
2,Hobbyist,Do you code as a hobby?
3,Age,What is your age (in years)? If you prefer not...
4,Age1stCode,At what age did you write your first line of c...
8,Country,Where do you live?
9,CurrencyDesc,Which currency do you use day-to-day? If your ...
10,CurrencySymbol,Which currency do you use day-to-day? If your ...
12,DatabaseWorkedWith,Which database environments have you done exte...
13,DevType,Which of the following describe you? Please se...


Save cleaned data files

In [23]:
df.to_csv(r'C:\\Users\\Jessica\\pj1_stackoverflow\\pj1-stackoverflow\\reduced_data.csv', index = False)
df_schem.to_csv(
    r'C:\\Users\\Jessica\\pj1_stackoverflow\\pj1-stackoverflow\\reduced_data_schema.csv', index = False)